# **Version 1**





данный код выгружает информацию о пользователях заданных групп, а после этого добавляет дополнительную информацию о них с помощью выгрузки информации из их личной страницы (данные находятся и объединяются с помощью user_id)

In [ ]:
token = 'hhh'
groups = ['towntravel', 'unknown_russia', "everydayfunday", "treking", "go_hiking_in_russia"]
path = r"/"
api = r"https://api.vk.com/method/groups.getMembers"
count = 100
 
fields = ['bdate', 'city', 'country', 'education', 'sex', 'relation']
 
def age(date_str):
    birthdate = datetime.datetime.strptime(date_str, '%d.%m.%Y')
    today = datetime.date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return(age)
 
data = {'First Name': [],
    'Last Name': [],
    'City': [],
    'Group':[]
    }
 
df = pd.DataFrame(data)
 
bebra_list = []
 
for group in groups:
    
    all_posts = []
    offset = 0
    while offset < 5000:
        
        response = requests.get(api,
                                    params={
                                        'access_token': token,
                                        'v': 5.131,
                                        'group_id': group,
                                        'fields': 'city',
                                        'offset' : offset,
                                        'count' : count
                                    }
                                    )
 
        data = response.json()['response']['items']
        all_posts.extend(data)
        bebra_list.extend(data)
        offset += 100
        
    for i in range(len(all_posts)):
        try:
            list = [all_posts[i]['first_name'], all_posts[i]['last_name'], all_posts[i]['city']['title'], group]
            df.loc[len(df)] = list
        except:
            list = [all_posts[i]['first_name'], all_posts[i]['last_name'], '', group]
            df.loc[len(df)] = list
            
api_2 = r"https://api.vk.com/method/users.get"
 
for i, post in enumerate(bebra_list):
  try:
      response2 = requests.get(api_2,
                          params={
                              'access_token': token,
                              'v': 5.131,
                              'fields': 'bdate',
                              'user_ids': np.int64(post["id"]),
                          }
                          )
 
      df.loc[i,'age'] = age(response2.json()['response'][0]['bdate'])
  except:
      pass
 
  try:
    response2 = requests.get(api_2,
                    params={
                        'access_token': token,
                        'v': 5.131,
                        'fields': 'sex',
                        'user_ids': np.int64(post["id"]),
                    }
                    )
 
    df.loc[i,'sex'] = np.where(response2.json()['response'][0]['sex'] == 2, 'male', 'female')
 
  except:
      pass
 
df.to_csv(path, index=False)
df = pd.read_csv(path)


# **Version 2**

Данный код выгружает все посты заданных групп по ключевым словам (query), а также информацию о пользователях которые сделали данные посты

In [ ]:
def get_posts():
    
    token = 'hhh'
    # domain = ['towntravel', 'unknown_russia', "everydayfunday", "treking", "go_hiking_in_russia"]
    
    domain = ['towntravel', 'unknown_russia', "everydayfunday", "treking", "go_hiking_in_russia", "palatka_tru", "ryukzak_za_plechi", "bro_dya_g","lup_stories", "katontour", "gotovimvpohode", "go__tours"]
    
    path = r"/"
    api_1 = r"https://api.vk.com/method/wall.search"
    query = ["Байкал", "Алтай", "Карелия", "Краснодар", "Туапсе", "Анапа", "Геленджик", "Санкт-Петербург"]
 
    api_2 = r"https://api.vk.com/method/users.get"
    count = 100
 
    all_posts = []
    
    # df = pd.read_csv(path)
    
    
    data = {
        # api 1
        'user_id': [],
        'date': [],
        'destination': [],
            
        # api 2  
        'sex': [],
        'city':[],
        'age':[]
        }
 
    df = pd.DataFrame(data)
    
    for public in domain:
        
        for word in query:
 
            offset = 0
 
            while offset < 1000:
 
                response = requests.get(api_1,
                                        params={
                                            'access_token': token,
                                            'v': 5.131,
                                            'domain': public,
                                            'query': word,
                                            'owners_only': 1,
                                            'count': count,
                                            'offset': offset
                                        }
                                        )
                
                data = response.json()["response"]["items"]
                all_posts.extend(data)
                offset += 100
                
                
 
            for i, post in enumerate(all_posts):
                
                try:
                    # df.loc[i] = [np.int64(post["signer_id"]), datetime.date.fromtimestamp(post["date"]), word]
                    df.loc[i, 'user_id'] = np.int64(post["signer_id"])
                    df.loc[i, 'date'] = datetime.date.fromtimestamp(post["date"])
                    df.loc[i, 'destination'] = word
                    
                except:
                    pass
 
                else:
                    try:
                        response2 = requests.get(api_2,
                                        params={
                                            'access_token': token,
                                            'v': 5.131,
                                            'fields': 'sex',
                                            'user_ids': np.int64(post["signer_id"]),
                                        }
                                        )
                    
                        df.loc[i,'sex'] = np.where(response2.json()['response'][0]['sex'] == 2, 'male', 'female')
                    
                    except:
                        pass
 
                    
                    try:
                        response2 = requests.get(api_2,
                                            params={
                                                'access_token': token,
                                                'v': 5.131,
                                                'fields': 'bdate',
                                                'user_ids': np.int64(post["signer_id"]),
                                            }
                                            )
 
                        df.loc[i,'age'] = age(response2.json()['response'][0]['bdate'])
                    except:
                        pass
                    
                    try:
                        response2 = requests.get(api_2,
                                            params={
                                                'access_token': token,
                                                'v': 5.131,
                                                'fields': 'city',
                                                'user_ids': np.int64(post["signer_id"]),
                                            }
                                            )
 
                        df.loc[i,'city'] = response2.json()['response'][0]['city']['title']
                        
                        
                    except:
                        pass
                    
                    
                    
                    
            all_posts = []
    
    df.to_csv(path, index=False)
    df = pd.read_csv(path)
    
    return(df)
